<a href="https://colab.research.google.com/github/Sv99t/hw_func/blob/main/HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U kaggle_environments

In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

# Агент 001 (камень)

In [ ]:
%%writefile rock_agent.py

#0 - rock
#1 - paper
#2 - scissors
def rock_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-98.0, 98.0]]

# Агент 002 (бумага)

In [ ]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["paper_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-97.0, 97.0]]

# Агент 003 (ножницы)

In [ ]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["scissors_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-99.0, 99.0]]

# Агент 004 (рандом)

In [ ]:
%%writefile random_opponent.py

import random

def random_opponent(observation, configuration):
    return random.randrange(0, configuration.signs)

Overwriting random_opponent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["random_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

# Агент 005 (все по очереди)

In [ ]:
%%writefile sequence_agent.py

counter = -1

def sequence(observation, configuration):
    global counter
    counter += 1
    if counter % 3 == 0:
        return 0
    elif counter % 3 == 1:
        return 1
    else:
        return 2

Overwriting sequence_agent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["sequence_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

# Агент 006 (следующий)

In [ ]:
%%writefile copynext_opponent.py

import random

def copynext_opponent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

Overwriting copynext_opponent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["copynext_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[57.0, -57.0]]

# Агент 007 (предыдущий)

In [ ]:
%%writefile copyprevious_opponent.py

import random

def copyprevious_opponent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

Overwriting copyprevious_opponent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["copyprevious_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-99.0, 99.0]]

# Агент 008 (повторюшка)

In [ ]:
%%writefile copy_opponent.py

import random

def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["copy_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-21.0, 21.0]]

# Агент 009 (оппозиционер)

In [ ]:
%%writefile opposit_agent.py

import random

def opposit_agent(observation, configuration):
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return random.choice([1,2])
        elif observation.lastOpponentAction == 1:
            return random.choice([0,2])
        else:
            return random.choice([1,0])
    else:
        return random.randrange(0, configuration.signs)

Overwriting opposit_agent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["opposit_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

# Агент 010 (3 раза и все)

In [ ]:
%%writefile random3_opponent.py

import random

counter = 0
stable_sign = None

def random3_opponent(observation, configuration):
    global counter
    global stable_sign
    if observation.step == 0:
        stable_sign = random.randrange(0, configuration.signs)
    elif counter % 3 == 2:
        stable_sign = random.randrange(0, configuration.signs)
    counter += 1
    return stable_sign

Writing random3_opponent.py


In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["random3_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

# **Турнир**
*Составим список агентов, потом заставим их сыграть между собой по количеству times партий. Получившуюся статистику побед запишем в словарь и выведем в виде таблицы pandas*

In [ ]:
agents = ['rock_agent.py',
          'paper_agent.py',
          'scissors_agent.py',
          'random_opponent.py',
          'sequence_agent.py',
          'copynext_opponent.py',
          'copyprevious_opponent.py',
          'copy_opponent.py',
          'opposit_agent.py', 
          'random3_opponent.py']


rank = {agent: 0 for agent in agents}

for times in range(5):
# сколько раундов сыграет каждая пара
          
    for i in range(len(agents)):
        for j in range(i+1, len(agents)):
            e = evaluate(
            "rps", #environment to use - no need to change
            [agents[i], agents[j]], #agents to evaluate
            configuration={"episodeSteps": 100, 'tieRewardThreshold': 1} #number of episodes 
            )
            if e[0][0] > e[0][1]:
                rank[agents[i]] += 1
            elif e[0][0] < e[0][1]:
                rank[agents[j]] += 1

# **Результаты**

In [ ]:
rank_se = pd.Series(rank, name="Таблица по количеству побед").sort_values(ascending=False).to_frame()
rank_se

,Таблица по количеству побед
copynext_opponent.py,28
opposit_agent.py,24
paper_agent.py,23
random_opponent.py,21
rock_agent.py,20
scissors_agent.py,20
copy_opponent.py,18
copyprevious_opponent.py,17
random3_opponent.py,17
sequence_agent.py,10


# **Вывод**
### Как мы видим, по результатам состязаний победил тот Агент, который выбирал следующий предмет от того, которым пользовался в предыдущем ходу его опонент